## 1. 데이터 불러오기
- data 폴더 안에 있는 csv 파일을 DataFrame으로 불러옵시다.
- 읍면동별, 상세 업종별 카드 이용 데이터(2017년): jeju_card_region_2017.csv
- 읍면동별, 상세 업종별 카드 이용 데이터(2018년): jeju_card_region_2018.csv
- 읍면동 단위 내국인 유동인구 데이터: jeju_population.csv
## 2. 데이터 탐색 및 전처리
- 데이터 개수, 컬럼별 데이터 타입, 통계 정보, 결측값 존재 여부 등을 확인해 보세요.
- 각 컬럼이 어떤 값들로 이루어져 있는지 확인해 보세요.
- 2017년과 2018년 카드 데이터를 하나로 합쳐 주세요.
## 3. 데이터 분석
- 이용금액과 이용자수가 많은 업종과 지역 Top 10을 각각 뽑아 보세요.
- 인당 이용금액이 많은 지역 Top 10도 뽑아 봅시다.
- 각 지역에서 어떤 업종들이 활성화되어 있는지도 함께 확인해 보세요.
- 카드 매출액 데이터와 유동인구 데이터를 하나로 합쳐서 보면 되겠죠? 어떻게 데이터를 합치는 게 좋을지 한번 고민해 보세요.

In [231]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import platform
%matplotlib inline

In [232]:
# 사용자 운영체제 확인
platform.system()
# 운영체제별 한글 폰트 설정
if platform.system() == 'Darwin': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')

In [233]:
# 데이터 불러오기
jeju_reg_17_df = pd.read_csv('data/jeju_card_region_2017.csv')
jeju_reg_18_df = pd.read_csv('data/jeju_card_region_2018.csv')
jeju_pop_df = pd.read_csv('data/jeju_population.csv')

In [234]:
# 데이터 개수 확인
print(jeju_reg_17_df.shape) 
print(jeju_reg_18_df.shape)
print(jeju_pop_df.shape)   

(26968, 7)
(27183, 7)
(527026, 6)


In [235]:
# 17년도 통계정보 확인
jeju_reg_17_df.describe(include='all')

,연월,시군구명,읍면동명,업종명,성별,이용자수,이용금액
count,26968,26968,26968,26968,26968,26968.000,26968.000
unique,12,2,43,41,2,NaN,NaN
top,2017-11-01,제주시,노형동,슈퍼마켓,남성,NaN,NaN
freq,2265,16076,795,1032,13518,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,1703.495,63231393.341
std,NaN,NaN,NaN,NaN,NaN,4313.161,198798394.063
min,NaN,NaN,NaN,NaN,NaN,1.000,10.000
25%,NaN,NaN,NaN,NaN,NaN,43.000,2635000.000
50%,NaN,NaN,NaN,NaN,NaN,281.000,12125170.000
75%,NaN,NaN,NaN,NaN,NaN,1333.250,49059281.000


In [236]:
# 고정 소수점 표기법으로 변경
pd.options.display.float_format = '{:.3f}'.format

In [237]:
# 2017년과 2018년 데이터 비교 및 전처리
def print_unique_values(df):
    object_columns = df.columns[df.dtypes == 'object']
    for col in object_columns:
        print(f'{col} 컬럼의 unique 값 개수: {df[col].nunique()}')
        print(sorted(df[col].unique()), '\n')

In [238]:
print_unique_values(jeju_reg_17_df)

연월 컬럼의 unique 값 개수: 12
['2017-01-01', '2017-02-01', '2017-03-01', '2017-04-01', '2017-05-01', '2017-06-01', '2017-07-01', '2017-08-01', '2017-09-01', '2017-10-01', '2017-11-01', '2017-12-01'] 

시군구명 컬럼의 unique 값 개수: 2
['서귀포시', '제주시'] 

읍면동명 컬럼의 unique 값 개수: 43
['건입동', '구좌읍', '남원읍', '노형동', '대륜동', '대정읍', '대천동', '도두동', '동홍동', '봉개동', '삼도1동', '삼도2동', '삼양동', '서홍동', '성산읍', '송산동', '아라동', '안덕면', '애월읍', '연동', '영천동', '예래동', '오라동', '외도동', '용담1동', '용담2동', '우도면', '이도1동', '이도2동', '이호동', '일도1동', '일도2동', '정방동', '조천읍', '중문동', '중앙동', '천지동', '추자면', '표선면', '한경면', '한림읍', '화북동', '효돈동'] 

업종명 컬럼의 unique 값 개수: 41
['건강보조식품 소매업', '골프장 운영업', '과실 및 채소 소매업', '관광 민예품 및 선물용품 소매업', '그외 기타 분류안된 오락관련 서비스업', '그외 기타 스포츠시설 운영업', '그외 기타 종합 소매업', '기타 갬블링 및 베팅업', '기타 대형 종합 소매업', '기타 수상오락 서비스업', '기타 외국식 음식점업', '기타 주점업', '기타음식료품위주종합소매업', '내항 여객 운송업', '마사지업', '면세점', '버스 운송업', '비알콜 음료점업', '빵 및 과자류 소매업', '서양식 음식점업', '수산물 소매업', '슈퍼마켓', '스포츠 및 레크레이션 용품 임대업', '여관업', '여행사업', '욕탕업', '육류 소매업', '일반유흥 주점업', '일식 음식점업', '자동차 임대업', '전시 및 행사 

In [239]:
print_unique_values(jeju_reg_18_df)

연월 컬럼의 unique 값 개수: 12
['2018-01-01', '2018-02-01', '2018-03-01', '2018-04-01', '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01', '2018-09-01', '2018-10-01', '2018-11-01', '2018-12-01'] 

시군구명 컬럼의 unique 값 개수: 2
['서귀포시', '제주시'] 

읍면동명 컬럼의 unique 값 개수: 43
['건입동', '구좌읍', '남원읍', '노형동', '대륜동', '대정읍', '대천동', '도두동', '동홍동', '봉개동', '삼도1동', '삼도2동', '삼양동', '서홍동', '성산읍', '송산동', '아라동', '안덕면', '애월읍', '연동', '영천동', '예래동', '오라동', '외도동', '용담1동', '용담2동', '우도면', '이도1동', '이도2동', '이호동', '일도1동', '일도2동', '정방동', '조천읍', '중문동', '중앙동', '천지동', '추자면', '표선면', '한경면', '한림읍', '화북동', '효돈동'] 

업종명 컬럼의 unique 값 개수: 41
['건강보조식품 소매업', '골프장 운영업', '과실 및 채소 소매업', '관광 민예품 및 선물용품 소매업', '그외 기타 분류안된 오락관련 서비스업', '그외 기타 스포츠시설 운영업', '그외 기타 종합 소매업', '기타 대형 종합 소매업', '기타 수상오락 서비스업', '기타 외국식 음식점업', '기타 주점업', '기타음식료품위주종합소매업', '내항 여객 운송업', '마사지업', '면세점', '버스 운송업', '비알콜 음료점업', '빵 및 과자류 소매업', '서양식 음식점업', '수산물 소매업', '슈퍼마켓', '스포츠 및 레크레이션 용품 임대업', '여관업', '여행사업', '욕탕업', '육류 소매업', '일반유흥 주점업', '일식 음식점업', '자동차 임대업', '전시 및 행사 대행업', '정기 항공 운송업

In [240]:
# 공통되지 않은 업종 찾기 
for item in jeju_reg_17_df['업종명'].unique():
    if item not in jeju_reg_18_df['업종명'].unique():
        print(f'2017년에만 있는 값: {item}')
        
for item in jeju_reg_18_df['업종명'].unique():
    if item not in jeju_reg_17_df['업종명'].unique():
        print(f'2018년에만 있는 값: {item}')

2017년에만 있는 값: 기타 갬블링 및 베팅업
2018년에만 있는 값: 택시 운송업


In [241]:
# 공통되지 않은 업종 제거
jeju_reg_17_df = jeju_reg_17_df[jeju_reg_17_df['업종명'] != '기타 갬블링 및 베팅업']
jeju_reg_18_df = jeju_reg_18_df[jeju_reg_18_df['업종명'] != '택시 운송업']

In [242]:
# 데이터 합치기
jeju_reg_df = pd.concat([jeju_reg_17_df, jeju_reg_18_df])

In [243]:
# 연월 컬럼 확인
jeju_reg_df['연월'].unique()

array(['2017-01-01', '2017-02-01', '2017-03-01', '2017-04-01',
       '2017-05-01', '2017-06-01', '2017-07-01', '2017-08-01',
       '2017-09-01', '2017-10-01', '2017-11-01', '2017-12-01',
       '2018-01-01', '2018-02-01', '2018-03-01', '2018-04-01',
       '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01',
       '2018-09-01', '2018-10-01', '2018-11-01', '2018-12-01'],
      dtype=object)

In [244]:
# 연월 컬럼 전처리
jeju_reg_df['연월'] = jeju_reg_df['연월'].str[:7]
jeju_reg_df

,연월,시군구명,읍면동명,업종명,성별,이용자수,이용금액
0,2017-01,서귀포시,남원읍,건강보조식품 소매업,남성,11,137500
1,2017-01,서귀포시,천지동,건강보조식품 소매업,여성,61,12334400
2,2017-01,서귀포시,대정읍,기타음식료품위주종합소매업,남성,555,17301300
3,2017-01,서귀포시,대정읍,기타 주점업,남성,324,71843080
4,2017-01,서귀포시,대정읍,기타 외국식 음식점업,남성,40,971000
...,...,...,...,...,...,...,...
27178,2018-12,제주시,오라동,중식 음식점업,여성,110,3043250
27179,2018-12,제주시,연동,스포츠 및 레크레이션 용품 임대업,여성,2317,365724920
27180,2018-12,제주시,연동,호텔업,여성,810,232661992
27181,2018-12,제주시,오라동,기타음식료품위주종합소매업,남성,25,1225510


In [245]:
# groupby로 업종명으로 합치기
groupby_sector = jeju_reg_df.groupby('업종명').sum(numeric_only=True)
groupby_sector

,이용자수,이용금액
업종명,,
건강보조식품 소매업,43738,7042351269
골프장 운영업,454602,63978385373
과실 및 채소 소매업,1088336,70647018956
관광 민예품 및 선물용품 소매업,611692,20571366644
그외 기타 분류안된 오락관련 서비스업,2361,116100063
그외 기타 스포츠시설 운영업,18761,1191623694
그외 기타 종합 소매업,920327,61397685585
기타 대형 종합 소매업,3026355,176181070237
기타 수상오락 서비스업,1188,214491300


In [246]:
# 이용금액이 많은 업종 Top 10
groupby_sector.sort_values(by='이용금액', ascending=False).head(10)

,이용자수,이용금액
업종명,,
한식 음식점업,18828766,764903591034
슈퍼마켓,11427536,303567124207
차량용 주유소 운영업,5683264,302282701948
면세점,2122760,216084015155
체인화 편의점,22577070,208385597308
기타 대형 종합 소매업,3026355,176181070237
일반유흥 주점업,541603,116184655729
스포츠 및 레크레이션 용품 임대업,991236,112011498985
정기 항공 운송업,640930,102803758104


In [247]:
# 이용자수가 많은 업종 Top 10
groupby_sector.sort_values(by='이용자수', ascending=False).head(10)

,이용자수,이용금액
업종명,,
체인화 편의점,22577070,208385597308
한식 음식점업,18828766,764903591034
슈퍼마켓,11427536,303567124207
차량용 주유소 운영업,5683264,302282701948
서양식 음식점업,3483836,82613944504
빵 및 과자류 소매업,3295046,45801876571
기타 대형 종합 소매업,3026355,176181070237
비알콜 음료점업,2990107,32196462878
"피자, 햄버거, 샌드위치 및 유사 음식점업",2963257,38337318851


In [248]:
# groupby로 지역명으로 합치기
groupby_region = jeju_reg_df.groupby('읍면동명').sum(numeric_only=True)
groupby_region

,이용자수,이용금액
읍면동명,,
건입동,1247860,46077137130
구좌읍,1593134,48035516926
남원읍,1209827,52349473146
노형동,7642921,245644045189
대륜동,1735095,67069487603
대정읍,1715861,56164010423
대천동,1221892,33847001011
도두동,741136,29033944428
동홍동,1959703,65748200288


In [249]:
# 이용금액이 많은 지역 Top 10
groupby_region.sort_values(by='이용금액', ascending=False).head(10)

,이용자수,이용금액
읍면동명,,
연동,7100596,340461915654
용담2동,5820083,289912196043
노형동,7642921,245644045189
송산동,5468346,205357714141
이도2동,6742075,201304448132
천지동,3683576,132477575549
애월읍,3397281,120988320522
중앙동,3053162,118137020208
정방동,3053162,118137020208


In [250]:
# 이용자수가 많은 지역 Top 10
groupby_region.sort_values(by='이용자수', ascending=False).head(10)

,이용자수,이용금액
읍면동명,,
노형동,7642921,245644045189
연동,7100596,340461915654
이도2동,6742075,201304448132
용담2동,5820083,289912196043
송산동,5468346,205357714141
천지동,3683576,132477575549
아라동,3653006,80280971143
애월읍,3397281,120988320522
일도2동,3145294,106631315225


In [251]:
# 인당 이용금액 컬럼 생성
groupby_region['인당이용금액'] = groupby_region['이용금액']/groupby_region['이용자수']
groupby_region

,이용자수,이용금액,인당이용금액
읍면동명,,,
건입동,1247860,46077137130,36924.925
구좌읍,1593134,48035516926,30151.586
남원읍,1209827,52349473146,43270.214
노형동,7642921,245644045189,32140.074
대륜동,1735095,67069487603,38654.649
대정읍,1715861,56164010423,32732.261
대천동,1221892,33847001011,27700.485
도두동,741136,29033944428,39174.921
동홍동,1959703,65748200288,33550.084


In [252]:
# 인당 이용금액이 많은 지역 Top 10
groupby_region.sort_values(by='인당이용금액', ascending=False).head(10)

,이용자수,이용금액,인당이용금액
읍면동명,,,
예래동,1259725,72430813294,57497.321
영천동,652686,35970307258,55111.198
용담2동,5820083,289912196043,49812.382
연동,7100596,340461915654,47948.358
일도1동,1161659,55313193619,47615.689
삼도2동,1436282,63122056691,43948.233
남원읍,1209827,52349473146,43270.214
이도1동,1082146,45553719645,42095.724
중문동,1599439,66174146830,41373.348


In [253]:
# 지역명과 업종명 groupby
groupby_region_sector = jeju_reg_df.groupby(['읍면동명', '업종명']).sum(numeric_only=True).reset_index()
groupby_region_sector

,읍면동명,업종명,이용자수,이용금액
0,건입동,건강보조식품 소매업,49,4587000
1,건입동,과실 및 채소 소매업,2501,391500342
2,건입동,관광 민예품 및 선물용품 소매업,3502,75545640
3,건입동,그외 기타 종합 소매업,3,275000
4,건입동,기타 외국식 음식점업,5293,181635728
...,...,...,...,...
1227,효돈동,차량용 주유소 운영업,80406,3618787777
1228,효돈동,체인화 편의점,110730,1044920210
1229,효돈동,"피자, 햄버거, 샌드위치 및 유사 음식점업",14501,122548910
1230,효돈동,한식 음식점업,50203,2056391378


In [254]:
# 각 지역에서 어떤 업종들이 활성화되어 있는지 확인
for region in groupby_region_sector['읍면동명'].unique():
    region_data = groupby_region_sector[groupby_region_sector['읍면동명'] == region]
    region_data = region_data.sort_values(by='이용자수', ascending=False).head(10)
    print(region_data)

   읍면동명                      업종명    이용자수         이용금액
24  건입동                  체인화 편의점  289677   2507213130
26  건입동                  한식 음식점업  234816  10667669485
12  건입동                     슈퍼마켓  204619   8937264501
23  건입동              차량용 주유소 운영업  120684   5936930821
10  건입동                 서양식 음식점업   94057   1535173042
7   건입동                      면세점   49248   3215267313
22  건입동               차량용 가스 충전업   42591   1439888624
9   건입동              빵 및 과자류 소매업   35243    715096540
25  건입동  피자, 햄버거, 샌드위치 및 유사 음식점업   29603    404126400
8   건입동                 비알콜 음료점업   27725    275428460
   읍면동명            업종명    이용자수         이용금액
57  구좌읍        한식 음식점업  473270  16214770362
55  구좌읍        체인화 편의점  308625   2922152476
41  구좌읍       서양식 음식점업  190175   3653335110
54  구좌읍    차량용 주유소 운영업  159671   8784917013
38  구좌읍  기타음식료품위주종합소매업  133252   4408931727
43  구좌읍           슈퍼마켓   83612   1810691155
51  구좌읍    전시 및 행사 대행업   58423   1299330400
39  구좌읍       비알콜 음료점업   53047    922838700
50  구좌읍   

In [255]:
# 유동인구 연월일 컬럼 확인
jeju_pop_df['연월일'].unique()

array([20170101, 20170102, 20170103, 20170104, 20170105, 20170106,
       20170107, 20170108, 20170109, 20170110, 20170111, 20170112,
       20170113, 20170114, 20170115, 20170116, 20170117, 20170118,
       20170119, 20170120, 20170121, 20170122, 20170123, 20170124,
       20170125, 20170126, 20170127, 20170128, 20170129, 20170130,
       20170131, 20170201, 20170202, 20170203, 20170204, 20170205,
       20170206, 20170207, 20170208, 20170209, 20170210, 20170211,
       20170212, 20170213, 20170214, 20170215, 20170216, 20170217,
       20170218, 20170219, 20170220, 20170221, 20170222, 20170223,
       20170224, 20170225, 20170226, 20170227, 20170228, 20170301,
       20170302, 20170303, 20170304, 20170305, 20170306, 20170307,
       20170308, 20170309, 20170310, 20170311, 20170312, 20170313,
       20170314, 20170315, 20170316, 20170317, 20170318, 20170319,
       20170320, 20170321, 20170322, 20170323, 20170324, 20170325,
       20170326, 20170327, 20170328, 20170329, 20170330, 20170

In [256]:
# 유동인구 데이터 전처리
jeju_pop_df['연월일'] = jeju_pop_df['연월일'].astype('string')
jeju_pop_df['연월'] = jeju_pop_df['연월일'].str[:4] + '-' + jeju_pop_df['연월일'].str[4:6]
jeju_pop_df['성별'] = jeju_pop_df['성별'] + '성'
jeju_pop_df.head()

,연월일,시군구명,읍면동명,성별,연령대,방문인구,연월
0,20170101,제주시,한림읍,남성,40대,19424,2017-01
1,20170101,제주시,애월읍,여성,20대,27747,2017-01
2,20170101,제주시,구좌읍,남성,70대,3459,2017-01
3,20170101,제주시,조천읍,남성,40대,36695,2017-01
4,20170101,제주시,한경면,남성,70대,1174,2017-01


In [259]:
# 카드 매출 데이터 컬럼 확인
jeju_reg_df.head()

,연월,시군구명,읍면동명,업종명,성별,이용자수,이용금액
0,2017-01,서귀포시,남원읍,건강보조식품 소매업,남성,11,137500
1,2017-01,서귀포시,천지동,건강보조식품 소매업,여성,61,12334400
2,2017-01,서귀포시,대정읍,기타음식료품위주종합소매업,남성,555,17301300
3,2017-01,서귀포시,대정읍,기타 주점업,남성,324,71843080
4,2017-01,서귀포시,대정읍,기타 외국식 음식점업,남성,40,971000


In [257]:
# 카드 매출 데이터와 맞게 그룹화
groupby_pop = jeju_pop_df.groupby(['연월', '시군구명', '읍면동명', '성별']).sum(numeric_only=True).reset_index()
groupby_pop.head()

,연월,시군구명,읍면동명,성별,방문인구
0,2017-01,서귀포시,남원읍,남성,2844830
1,2017-01,서귀포시,남원읍,여성,2430535
2,2017-01,서귀포시,대륜동,남성,2643763
3,2017-01,서귀포시,대륜동,여성,2293711
4,2017-01,서귀포시,대정읍,남성,2346595


In [258]:
# 카드 매출액 데이터와 유동인구 데이터 합치기
jeju_df = pd.merge(jeju_reg_df, groupby_pop, how='left', on=['연월', '시군구명', '읍면동명', '성별'])
jeju_df.head()

,연월,시군구명,읍면동명,업종명,성별,이용자수,이용금액,방문인구
0,2017-01,서귀포시,남원읍,건강보조식품 소매업,남성,11,137500,2844830.000
1,2017-01,서귀포시,천지동,건강보조식품 소매업,여성,61,12334400,685754.000
2,2017-01,서귀포시,대정읍,기타음식료품위주종합소매업,남성,555,17301300,2346595.000
3,2017-01,서귀포시,대정읍,기타 주점업,남성,324,71843080,2346595.000
4,2017-01,서귀포시,대정읍,기타 외국식 음식점업,남성,40,971000,2346595.000
